# Linearize Datacolor .txt Files
This notebook will read Datacolor .txt files generated from reading a test chart, determine the resulting curve, and generate a smoothed linearization curve.

### Prior to Running Any Test Charts
1. Make a spectro calibration print.  Expose a piece of coated paper.  Cover up half of it with something completely opaque.  The other half should be covered with a blank piece of transparency.
1. Read the dark areas and find the darkest value you can.  Write it down - you'll be entering this number below as the **calib_darkest**.
1. Read the coated but unexposed areas and find the lightest value you can.  Write it down - you'll be entering this number below as the **calib_brightest**

You only need to do this step one time per process type.  After you enter these values, save the notebook so you can re-use these values for every subsequent test for that process type.  You may want to write the values you read on the paper itself in case you change this notebook for another type of process.

### Datacolor .txt Files
This notebook processes the .txt files generated by Spyder Print's Measurement function.  These files are merely space-delimited L\*a\*b files, where each line is one measurement and the L, a, and b values are separated by space characters.  The measurements should be in order in the file from the lightest patch to the darkest patch.

**WHEN READING YOUR TEST CHARTS, READ FROM THE LIGHT PATCHES TO THE DARK PATCHES, IN ORDER!**

Note that if you are using a spectrocolorimeter other than a Datacolor device, you can still produce this file, providing you can get L\*a\*b values out of it.  The formatting of the .txt file must conform to the description above, but the notebook doesn't care what device produced the values.

### Preparing Test Chart Negatives
When making your negative, **ensure that you add a patch of pure black** after you have inverted the chart image.  This patch must be as black as your printer can produce, so the paper under it is coated, but completely unexposed.  You will be reading this resulting patch as your lightest reading to make sure the values in the .txt file can be normalized to correspond to other tests.  Basically you are determining the lightest possible value your coated paper can produce.

### Reading the Test Charts
1. Read the test chart in Spyder Print's Measurement function.  Do this at least 3 times, producing 3 separate .txt files. If you want, you can do this more times for a better average per patch.  For each patch, the values for all of these files will be averaged to minimize noise and inconsistencies. 
1. Read the white patch you added to your negative.  This needs to be coated, but un-exposed.  Write this number down - you'll be entering it when prompted.
1. Read the darkest part of your print, preferably in a location where the film covered the paper, but had no ink density.  Write this number down - you'll be entering it when prompted.

### How To Use This Notebook
The code in this notebook is organized into cells.  Each cell can be run independently, providing all prerequisite imports have been done and all variables exist.  However, **it is recommended that you run all code cells in sequence.** 

**IMPORTANT:** As mentioned above, you will need to provide 4 values in the [Enter the calibration and test chart brightest and darkest values](#Enter-the-calibration-and-test-chart-brightest-and-darkest-values) cell.  **Do not forget to do this!**

The exceptions to this normal operation are that you may wish to run the [Mirror and Smooth](#Mirror-and-Smooth) cell more than once, providing different smoothing values until you are satisfied with the results.  Once you are happy with the resulting curve, you can then move on to the [Save the New Curve to a .cube File](#Save-the-New-Curve-to-a-.cube-File) cell to save the new .cube file.

# Enter the calibration and test chart brightest and darkest values

In [ ]:
# the following are values from the spectro calibration test print
calib_brightest = 96.5
calib_darkest = 1.5

# Import libraries

In [ ]:
import os
import sys
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate
from tkinter import Tk
from tkinter import filedialog
from tkinter import simpledialog

# Get the paths to the .txt files

In [ ]:
root = Tk()
root.wm_attributes('-topmost', 1)
root.withdraw()
txt_files = filedialog.askopenfilenames(title="Datacolor Measurement .txt file(s)", filetypes=[("TXT files", "*.txt")])

# Read the .txt files and average the values per cell

In [ ]:
dfs = []
for txt in txt_files:
    df = pd.read_csv(txt, header=None, usecols=[0], sep='\t')
    dfs.append(df)
# rename the columns in each dataframe
for i in range(len(dfs)):
    dfs[i].rename(columns={0:"Text_File_{}".format(i+1)}, inplace=True)
# make the value steps
value_steps = []
num_steps = len(dfs[0].index)
step_increment = 1.0 / (num_steps - 1)
min_val = 0
for i in range(num_steps):
    value_steps.append(round(min_val + (i * step_increment), 4) * 100)
# check to see if the value steps' order matches the first dataframe
mindf = dfs[0][['Text_File_1']].idxmin().values[0]
maxdf = dfs[0][['Text_File_1']].idxmax().values[0]
# create a dataframe using the value steps as the index
datacolor_df = pd.DataFrame({"Linear":value_steps}, index=value_steps)
# add the values from the TXT dataframes
column_names = []
for i in range(len(dfs)):
    col_vals = dfs[i]['Text_File_{}'.format(i+1)].tolist()
    # make sure the list values are increasing, not decreasing
    if col_vals[0] > col_vals[-1]:
        col_vals.reverse()
    datacolor_df['Text_File_{}'.format(i+1)] = col_vals
    column_names.append('Text_File_{}'.format(i+1))
# average the text file columns
datacolor_df["Avg_TXT_values"] = datacolor_df[column_names].mean(axis=1)

# Inspect the values
The cell below prints out all of the values so you can inspect them.  If you don't care to see them, you can skip this cell.

In [ ]:
datacolor_df.head(num_steps)

In [ ]:
def interpolate(xval, df, xcol, ycol):
    """
    This function returns an interpolated value from a dataframe at a specified X value.
    
    Inputs
    ------
    xval: numeric - the X value at which you wish to get the interpolated Y value
    df: pandas dataframe - the dataframe containing the data to be interpolated
    xcol: string - the name of the column in the dataframe containing X values
    ycol: string - the name of the column in the dataframe containing Y values
    
    Returns
    -------
    (interpolated_value, X, Y)
    """
    return np.interp([xval], df[xcol], df[ycol])


def perpendicular_dist_from_linear(pt_x, pt_y):
    linear_x = (pt_x + pt_y) / 2.0
    dist_from_linear = math.dist([linear_x, linear_x], [pt_x, pt_y])
    if pt_y < linear_x:
        dist_from_linear = -1.0 * dist_from_linear
    return [linear_x, dist_from_linear]

# Normalize the data so it falls within the calibrated range

In [ ]:
lightest_reading = simpledialog.askfloat(title="Lightest Measured Value", prompt="Enter the value of the coated but unexposed patch you read from your test.", initialvalue=calib_brightest)
darkest_reading = simpledialog.askfloat(title="Darkest Measured Value", prompt="Enter the value from fully exposed coated paper.", initialvalue=calib_darkest)
# Normalize - scale the values to fit within the calibrated values
dark_correct = calib_darkest - darkest_reading
new_lightest = lightest_reading + dark_correct
light_scaling = 100 / new_lightest
datacolor_df['Avg_TXT_values'] = (datacolor_df['Avg_TXT_values'] + dark_correct)
datacolor_df['Avg_TXT_values'] = (datacolor_df['Avg_TXT_values'] * light_scaling)

# Mirror and Smooth

In [ ]:
# mirror the values around the diagonal
perp_dists = {}
for i, row in datacolor_df.iterrows():
    the_x, the_dist = perpendicular_dist_from_linear(row['Linear'], row['Avg_TXT_values'])
    perp_dists[the_x] = the_dist
perp_vals = []
new_y_vals = []
for line_val in perp_dists.keys():
    dist_val = perp_dists[line_val]
    dist_sqrt = abs(dist_val) / math.sqrt(2.0)
    if dist_val < 0:
        perp_vals.append(line_val - dist_sqrt)
        new_y_vals.append(line_val + dist_sqrt)
    else:
        perp_vals.append(line_val + dist_sqrt)
        new_y_vals.append(line_val - dist_sqrt)
mirror_df = pd.DataFrame({"Line_Vals":perp_vals, "Mirror_Values": new_y_vals}, index=perp_vals)
# interpolate the data so it can be added to the original dataframe
new_curve_vals = []
for i in value_steps:
    ival = interpolate(i, mirror_df,'Line_Vals', 'Mirror_Values')
    new_curve_vals.append(ival[0])
datacolor_df['Mirrored_TXT_Values'] = new_curve_vals
# smooth the mirrored values
smoothing_factor_value = simpledialog.askinteger(title="Smoothing Factor", prompt="Enter a smoothing factor.\nBigger numbers mean a smoother result.\nValues between 20 and 150 are most common.", initialvalue=50)
if smoothing_factor_value == None:
    smoothing_factor_value = 50
avg_spline = scipy.interpolate.UnivariateSpline(datacolor_df['Linear'], datacolor_df['Mirrored_TXT_Values'], k=5)
avg_spline.set_smoothing_factor(smoothing_factor_value)
# get the smoothed readings, and make sure they're strictly increasing
smoothed_vals = avg_spline(datacolor_df['Linear']).tolist()
output_vals = []
previous_val = -1
for val in smoothed_vals:
    if val > previous_val:
        output_vals.append(val)
        previous_val = val
    else:
        output_vals.append(previous_val)
# add the values to the dataframe
datacolor_df['Linearized_Curve'] = output_vals

datacolor_df[['Linear','Avg_TXT_values', 'Mirrored_TXT_Values','Linearized_Curve']].plot(figsize=(7,7), grid=True)
plt.show()

# Save the New Curve to a .cube File

In [ ]:
# convert the curve so it falls within 0.0 and 1.0
converted_vals = [e/100.0 for e in datacolor_df['Linearized_Curve'].tolist()]

# get the output filename
cube_filename = simpledialog.askstring("Output .cube File Name", "Enter the name for the output .cube file (no .cube extension)")
cube_filename = cube_filename.replace(" ", "_")

# save the new curve to a .cube file
out_cube = os.path.join(os.path.dirname(txt_files[0]), "{}.cube".format(cube_filename))
with open(out_cube, 'w') as cube:
    cube.write("#Created by: Datacolor_Linearize Notebook\nTITLE  \"{}\"\nLUT_1D_SIZE {}\nDOMAIN_MIN 0.0 0.0 0.0\nDOMAIN_MAX 1.0 1.0 1.0\n#LUT data points\n".format(cube_filename, len(converted_vals)))
    line_template = "{:<08} {:<08} {:<08}\n"
    for row in converted_vals:
        if row < 0.0:
            val = 0.0
        elif row > 1.0:
            val = 1.0
        else:
            val = round(row, 6)
        cube.write(line_template.format(val, val, val))
    cube.write("#END data\n")
print("Saved {}".format(out_cube))